In [1]:
import pandas  as pd

from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

digits = load_digits()

In [2]:
clf = GridSearchCV(SVC(gamma='auto'), {
    'C': [1, 10, 20],
    'kernel': ['rbf', 'linear']
}, cv=5, return_train_score=False)
clf.fit(digits.data, digits.target)
pd.DataFrame(clf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.292038,0.035447,0.108018,0.013526,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.411111,0.450000,0.454039,0.448468,0.479109,0.448545,0.021761,6
1,0.023388,0.003977,0.005727,0.003343,1,linear,"{'C': 1, 'kernel': 'linear'}",0.963889,0.919444,0.966574,0.963788,0.924791,0.947697,0.020978,1
2,0.329540,0.055949,0.119678,0.027630,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.452778,0.469444,0.479109,0.479109,0.501393,0.476366,0.015784,4
3,0.045210,0.012656,0.006565,0.001992,10,linear,"{'C': 10, 'kernel': 'linear'}",0.963889,0.919444,0.966574,0.963788,0.924791,0.947697,0.020978,1
4,0.301713,0.066133,0.096853,0.020174,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.452778,0.469444,0.479109,0.479109,0.501393,0.476366,0.015784,4
5,0.035246,0.010383,0.007707,0.004273,20,linear,"{'C': 20, 'kernel': 'linear'}",0.963889,0.919444,0.966574,0.963788,0.924791,0.947697,0.020978,1


In [3]:
clf.best_params_, clf.best_score_

({'C': 1, 'kernel': 'linear'}, 0.9476973073351903)

In [4]:
rs = RandomizedSearchCV(SVC(gamma='auto'), {
        'C': [1,10,20],
        'kernel': ['rbf','linear']
    }, 
    cv=5, 
    return_train_score=False, 
    n_iter=2
)
rs.fit(digits.data, digits.target)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,20,rbf,0.476366
1,1,linear,0.947697


# Test of all classifiers

In [7]:
model_params = {
      'svm': {
            'model': SVC(gamma="auto"),
            'param': {
                'C': [1, 10, 20],
                'kernel': ['rbf', 'linear']
            }
      },
      'random_forest': {
            'model': RandomForestClassifier(),
            'param': {
                'n_estimators': [1, 5, 10]
            }
      },
      'decision_tree': {
            'model': DecisionTreeClassifier(),
            'param': {
                'criterion': ['gini', 'entropy']
            }
      },
      'log_reg': {
            'model': LogisticRegression(),
            'param': {
                  'C': [1, 5, 10]
            }
      },
      'gaus_nb': {
            'model': GaussianNB(),
            'param': {}
      },
      'milt_nb': {
            'model':MultinomialNB(),
            'param': {}
      }
}

In [8]:
scores = []

for model_name, model_param in model_params.items():
      clf = GridSearchCV(model_param['model'], model_param['param'], cv=5, return_train_score=False)
      clf.fit(digits.data, digits.target)
      scores.append({
            'model': model_name,
            'best_score': clf.best_score_,
            'best_param': clf.best_params_
      })

df = pd.DataFrame(scores, columns=['model', 'best_score', 'best_param'])
df

c:\Users\Артём\Desktop\лекции\Саморазвитие\ml_ind\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Артём\Desktop\лекции\Саморазвитие\ml_ind\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

,model,best_score,best_param
0,svm,0.947697,"{'C': 1, 'kernel': 'linear'}"
1,random_forest,0.899855,{'n_estimators': 10}
2,decision_tree,0.808044,{'criterion': 'entropy'}
3,log_reg,0.913765,{'C': 1}
4,gaus_nb,0.806928,{}
5,milt_nb,0.870350,{}
